## Code refactoring with ChatGPT

In [1]:
import inspect
import openai
import time
import matplotlib.pyplot as plt
import config

In [2]:
my_api_key = config.my_api_key

### Create ChatGPT class

#### We have to use suitable prompts and memory structure to achieve code refactoring

In [3]:
class ChatGPT:
    def __init__(self, api_key):
        self.engine = "text-davinci-003"
        self.openai = openai
        self.openai.api_key = api_key
        self.memory = ""
        self.reset_memory()
        
    def create_conversation(self, forget=True, remind_me=False, init_prompt=None):
        """Creates (forget=True) or continues (forget=False) a conversation interactable through prompts.
        args:
        forget: bool, if True: starts a new conversation, else: continues the conversation in memory
        remind_me: bool (has an effect only if forget is False), if True: prints conversation in memory, else: nothing
        init_prompt: str, initial prompt can be specified here instead of waiting for the input() function
        
        """
        
        if forget:
            self.reset_memory()
        else:
            if remind_me:
                print(self.memory)
                
        if init_prompt:
            user_input = init_prompt
        else:
            user_input = input("... ")
            
        # Want to quit?
        if user_input.upper() in ['Q', 'QUIT', 'DONE', 'FINISHED', 'BYE', 'EXIT']:
            return 0
        
        # Not quitting -- continue conversation
        else:
            self.remember(last_text = user_input, user=True)
            r = self.respond(prompt=self.memory)
            print(r)
            self.remember(last_text = r, user=False)
            self.create_conversation(forget=False, remind_me=False, init_prompt=None)
            
    def remember(self, last_text, user):
        """Appends the last_text to the memory in a suitable format."""
        if user:
            t = str(" I said: " + last_text)
        else:
            t = str(" You said: " + last_text)
        self.memory += t
        
    def reset_memory(self):
        """Resets the memory to a suitable form (not empty) to be able to start a new conversation or question."""
        self.memory = str("I would like you to finish or continue our conversation as a tutor, " +
                          "teacher or professor would in that situation, " +
                          "by giving the precise answer " +
                          "if the end of the conversation can be answered in a simple or precise way. " +
                          "Let's pretend that so far we discussed the following: ")
        
    def respond(self, prompt, max_tokens=100, n=1, stop=None, temperature=0.2):
        """Creates a response based on prompt."""
        completion = self.openai.Completion.create(engine=self.engine,
                                              prompt=prompt,
                                              max_tokens=max_tokens,
                                              n=n,
                                              stop=None,
                                              temperature=temperature)
        
        response = completion.choices[0].text
        return response
        
    def speed_up_code(self, f, refactor=False):
        """Inspects a function and comes up with the best way to speed it up.
        Either returns a suggestion or the refactored code itself.
        args:
        f: function, available in current memory (either imported or defined in the same environment)
        refactor: bool, if True: returns only the faster rewritten code, else: returns a human readable suggestion
        """
        if refactor:
            s = str("I would like to increase the speed of some code I wrote in python. " +
                "Could you figure out why my code is slow and rewrite it for me as per your number one suggestion? " +
                "I would only need the code at the end without an explanation. " +
                "Here is the code: \n")
            
        else:
            s = str("I would like to increase the speed of some code I wrote in python. " +
                "Could you figure out why my code is slow and give me one suggestion on how to speed it up? " +
                "Please give a short answer. "
                "Here is the code: \n")
        
        c = inspect.getsource(f)
        prompt = s + c
        self.remember(prompt, user=True)
        response = self.respond(prompt=prompt)
        self.remember(response, user=False)
        return response
        

### Create an inefficient function

In [4]:
def my_func(a):
    l = [1,2,3,4,5]
    
    for i in range(len(l)):
        l[i] = l[i] + a
    
    return l

### Initialize an instance of ChatGPT

In [5]:
c = ChatGPT(my_api_key)

### Call refactoring on function

In [6]:
print("Chat GPT's suggestion: ")
print("------------------------ ")

print(c.speed_up_code(my_func, refactor=True))

Chat GPT's suggestion: 
------------------------ 

def my_func(a):
    l = [1,2,3,4,5]
    
    l = [x + a for x in l]
    
    return l


### Continue discussion if one step was not sufficient

In [7]:
next_question = "Could you show me another way using numpy?"

In [8]:
discuss = c.create_conversation(forget=False, remind_me=False, init_prompt=next_question)



You could use numpy to increase the speed of your code by using the numpy.add() function. This function takes two arrays and adds the elements of one array to the elements of the other array. For example, if you have two arrays, A and B, you can add them together using the numpy.add() function like this: 

import numpy as np

C = np.add(A, B)

In your case, you
... Bye
